In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

x_train, x_valid = x_train_full[5000:], x_train_full[:5000]
y_train, y_valid = y_train_full[5000:], y_train_full[:5000]

x_mean = x_train.mean(axis=0, keepdims=True)
x_std = x_train.std(axis=0, keepdims=True) + 1e-7

x_train = (x_train - x_mean) / x_std
x_valid = (x_valid - x_mean) / x_std
x_test = (x_test - x_mean) / x_std

x_train = x_train[..., np.newaxis]
x_valid = x_valid[..., np.newaxis]
x_test = x_test[..., np.newaxis]

In [3]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, 
                        activation='relu', padding='SAME')

In [4]:
model = keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, 
                  input_shape=[28, 28, 1]),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),    
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation='softmax'),    
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        3200      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 256)         295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 256)         5

In [6]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='nadam', 
              metrics=['acc'])

history = model.fit(x_train, y_train, epochs=10, 
                    validation_data=(x_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 125s 72ms/step - loss: 0.7064 - acc: 0.7532 - val_loss: 0.3593 - val_acc: 0.8742
Epoch 2/10
1719/1719 [==============================] - 127s 74ms/step - loss: 0.4177 - acc: 0.8602 - val_loss: 0.3348 - val_acc: 0.8840
Epoch 3/10
1719/1719 [==============================] - 130s 76ms/step - loss: 0.3569 - acc: 0.8814 - val_loss: 0.3010 - val_acc: 0.8870
Epoch 4/10
1719/1719 [==============================] - 130s 76ms/step - loss: 0.3259 - acc: 0.8905 - val_loss: 0.3058 - val_acc: 0.8950
Epoch 5/10
1719/1719 [==============================] - 130s 76ms/step - loss: 0.3023 - acc: 0.8981 - val_loss: 0.2914 - val_acc: 0.8970
Epoch 6/10
1719/1719 [==============================] - 130s 76ms/step - loss: 0.2935 - acc: 0.9010 - val_loss: 0.2641 - val_acc: 0.9016
Epoch 7/10
1719/1719 [==============================] - 128s 74ms/step - loss: 0.2761 - acc: 0.9049 - val_loss: 0.2759 - val_acc: 0.9006
Epoch 8/10
1719/1719 [===================

In [7]:
score = model.evaluate(x_test, y_test)

313/313 [==============================] - 6s 18ms/step - loss: 0.3206 - acc: 0.8981
